In [ ]:
from skimage.feature import greycomatrix, greycoprops
from skimage import io, color, img_as_ubyte
import numpy as np
import pandas as pd

In [ ]:
# definiowanie zmiennych
n = 972 # liczba próbek
distances = [1,3,5] # odległości pikseli
angles = ['0','pi/4', 'pi/2', '3pi/4'] #kierunki
features = ['dissimilarity','correlation','contrast','energy','homogeneity','ASM'] #cechy
names = []

In [ ]:
# wypełnianie nazw kolumn
for a in range(0, len(features)):
    for b in range(0,len(distances)):
        for c in range(0,len(angles)):
            names.append( features[a] + '_' + str(distances[b]) + '_' + angles[c])


names.append('category')

In [ ]:
# sciezki do plikow
path_name = ['drzwi','plytka','roleta']
path = "C:\\Users\\zsuet\\Desktop\\studia\\2 stopień\\Programowanie w obliczeniach inteligentych\\zad3\\"

In [ ]:
# tworzenie macierzy zdarzeń dla każdej próbki dla każdej kategorii
for p_counter in range (0,len(path_name)): # pętla z różnymi kategoriami
    for iters in range (0,n): #pętla z różnymi próbkami w danej kategorii
        p = path + path_name[p_counter] + '\\fragments\\'
        img = io.imread(p + str(iters+1) + '.jpg') # ścieżka do poszczególnej próbki
        img = img_as_ubyte(color.rgb2gray(img)) # przekształcenie próbki do skali szarości
        img = np.digitize(img,[0,51,102,153,204,255]) # głębia jasności do 5 bitów
        glcm = greycomatrix(img, distances=distances, angles=[0,np.pi/4, np.pi/2, 3*np.pi/4],levels = img.max()+1)
        vector = []
        for i in range(0,len(features)): # pętla wyznaczająca cechy dla poszczególnej próbki
            arr = greycoprops(glcm,features[i])
            vector = np.append(vector,arr)


        if iters == 0: #zapisywanie cech do DF
            y = pd.DataFrame(vector).append(pd.DataFrame([path_name[p_counter]]))
        else:
            x = pd.DataFrame(vector).append(pd.DataFrame([path_name[p_counter]]))
            y = pd.concat([y,x],axis = 1)


    if p_counter == 0:
        df = y
    else:
        df = pd.concat([df,y],axis = 1)

In [ ]:
df = df.transpose() #transpopzycja DF
df.columns = names #zmiana nazw kolumn
rows = range (0,df.shape[0]) #zmiana nazw indexów
df.index = rows
df.to_csv(path + '\\result.csv') #zapis DF do pliku .csv